# Model generator

## Imports

In [2]:
# list of supported layers
from tensorflow.keras.layers import (
    Input,
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
    Lambda, # only for polynomial activation in the form of `Lambda(lambda x: x**2+x)`
    MaxPooling2D,
    ReLU,
    Softmax,
    )
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import tf2onnx
import onnxruntime as ort

## Dataset generator

In [20]:
X_train = np.random.randint(0, 255, size=(50, 32, 32, 3))
X_test = np.random.randint(0, 255, size=(10, 32, 32, 3))

y_train = np.random.randint(0, 4, size=(50, 1))
y_test = np.random.randint(0, 4, size=(10, 1))

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [21]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50, 32, 32, 3), (10, 32, 32, 3), (50, 4), (10, 4))

## Model definition

### Formulation

In [26]:
model = tf.keras.Sequential([
    #tf.keras.layers.Conv1D(filters=5, kernel_size=3, input_shape=(32, 32, 3)),
    #tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Conv2D(filters=30, kernel_size=2, ), 
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dense(12, activation='relu'),
    #tf.keras.layers.Dense(4, activation='relu'),
    #tf.keras.layers.AveragePooling2D(),
    #tf.keras.layers.MaxPooling2D(pool_size=(1, 1)),
    #tf.keras.layers.GlobalMaxPool2D(),
    #tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(name='layer_6'),
    tf.keras.layers.Dense(4, activation='softmax'),
])


In [27]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 32, 32, 8)         32        
                                                                 
 batch_normalization_3 (Batc  (None, 32, 32, 8)        32        
 hNormalization)                                                 
                                                                 
 layer_6 (Flatten)           (None, 8192)              0         
                                                                 
 dense_8 (Dense)             (None, 4)                 32772     
                                                                 
Total params: 32,836
Trainable params: 32,820
Non-trainable params: 16
_________________________________________________________________


### Training

In [28]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [29]:
model.fit(X_train, y_train, epochs=15, batch_size=16,  validation_data=(X_test, y_test))

Epoch 1/15
4/4 [==============================] - 3s 119ms/step - loss: 2.0006 - acc: 0.3000 - val_loss: 7.2763 - val_acc: 0.1000
Epoch 2/15
4/4 [==============================] - 0s 30ms/step - loss: 0.2165 - acc: 0.9800 - val_loss: 9.1187 - val_acc: 0.0000e+00
Epoch 3/15
4/4 [==============================] - 0s 28ms/step - loss: 0.0598 - acc: 1.0000 - val_loss: 9.4263 - val_acc: 0.0000e+00
Epoch 4/15
4/4 [==============================] - 0s 33ms/step - loss: 0.0435 - acc: 1.0000 - val_loss: 9.1949 - val_acc: 0.0000e+00
Epoch 5/15
4/4 [==============================] - 0s 28ms/step - loss: 0.0404 - acc: 1.0000 - val_loss: 8.7490 - val_acc: 0.0000e+00
Epoch 6/15
4/4 [==============================] - 0s 26ms/step - loss: 0.0390 - acc: 1.0000 - val_loss: 8.2382 - val_acc: 0.0000e+00
Epoch 7/15
4/4 [==============================] - 0s 27ms/step - loss: 0.0384 - acc: 1.0000 - val_loss: 7.7115 - val_acc: 0.0000e+00
Epoch 8/15
4/4 [==============================] - 0s 27ms/step - loss: 0

## Save

### Tensorflow model - h5

In [36]:
# Save tensorflow model
model.save('model.h5')

In [13]:
for layer in model.layers:
    print(layer.__class__.__name__, layer.get_config())
    try:
        print(layer.get_config()['function'])
    except:
        pass
    print(layer.get_input_shape_at(0),layer.get_output_shape_at(0))
    try:
        print(layer.get_weights()[0].shape)
        print(layer.get_weights()[1].shape)
    except:
        pass

Dense {'name': 'dense_3', 'trainable': True, 'batch_input_shape': (None, 20), 'dtype': 'float32', 'units': 8, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': {'class_name': 'L2', 'config': {'l2': 0.009999999776482582}}, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
(None, 20) (None, 8)
(20, 8)
(8,)
BatchNormalization {'name': 'batch_normalization_1', 'trainable': True, 'dtype': 'float32', 'axis': ListWrapper([1]), 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer': {'class_name': 'Ones', 'config': {}}, 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}}, 'moving_variance_initializer': {'class_name': 'Ones', 'config': {}}, 'beta_regularizer': None, 'gamma_regularizer':

### Onnx model - onnx

In [31]:
#Convert to Onnx
# Define the input signature for the conversion
input_signature = [tf.TensorSpec([None, 32, 32, 3], tf.float32, name="input")]

# Convert the model to ONNX
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, opset=13)

# Save the ONNX model to a file
with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## Input Json generator

In [32]:
import json


In [33]:
X = X_test[[0]]
y = model.predict(X)
y

1/1 [==============================] - 0s 129ms/step


array([[9.6215487e-01, 1.1055118e-03, 3.6557816e-02, 1.8187548e-04]],
      dtype=float32)

In [35]:
with open("input.json", "w") as f:
    json.dump({'in': X.flatten().tolist()}, f)